In [ ]:
### Step 1: Convert LAZ to LAS        ######

import arcpy
import os

# Input and output paths "Be careful with the path to your data"
input_folder = r"C:\Your path\CathedralPines_LAZ"
output_folder = r"C:\Your path\LASArcpy"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Convert LAZ to LAS and store paths
las_files = []

for file in os.listdir(input_folder):
    if file.lower().endswith(".laz"):
        laz_path = os.path.join(input_folder, file)

        # Use only required positional arguments
        arcpy.conversion.ConvertLas(
            laz_path,                # input LAS file
            output_folder,          # target folder
            "1.4",                 # 3 - LAS file version (e.g., "1.2")
            "6",       # 4 - Point format
            "No Compression",      # 5 - Compression method
            "", "", "", ""  # optional values: las options, out_las_dataset, define coordinate system, coordinate system
        )

        las_files.append(os.path.join(output_folder, file.replace(".laz", ".las")))

# Create LAS dataset "Mosaic all converted Las-tiles"
las_dataset = os.path.join(output_folder, "CathedralPines.lasd")
arcpy.management.CreateLasDataset(las_files, las_dataset)

print("LAS Dataset created at:", las_dataset)

In [ ]:

# Step 2: Create LAS dataset "Mosaic all converted Las-tiles" 2016

import arcpy
import os


output_folder = r"C:\Your path\CathedralPines_LAS"
las_dataset = os.path.join(output_folder, "CathedralPines_2016.lasd")
arcpy.management.CreateLasDataset(las_files, las_dataset)

print("LAS Dataset created at:", las_dataset)

In [ ]:
####   Step 3 : Extract the study area using the Shape file ####

import arcpy
from arcpy import env

env.workspace = 'C:/Your path/CathedralPines'

arcpy.ddd.ExtractLas(
    'C:/Your path/LASArcpy/CathedralPines.lasd',
    'C:/Your path/LASArcpy/Clipped_tiles',
    boundary='C:/Your path/CathPine_shape.shp',
    name_suffix='subset',
    remove_vlr=True,
    rearrange_points='REARRANGE_POINTS',
    out_las_dataset='Extract_CathedralPines_2016.lasd'
)

print("Points were clipped exactly to the shapefile boundary.")




In [ ]:
####  Step 4: DSM   ####

import arcpy

arcpy.env.workspace = r'C:/Your path/LASArcpy'
in_lasd = r'C:/Your path/LASArcpy/Extract_CathedralPines_2016.lasd'
out_raster = r'C:/Your path/DSM_CathedralPines_2016.tif'

# Using the older function syntax
arcpy.LasDatasetToRaster_conversion(
    in_lasd,
    out_raster,
    "ELEVATION",
    "BINNING MAXIMUM LINEAR",
    "FLOAT",
    "CELLSIZE",
    0.3,
    1)

print("DSM created successfully.")


In [ ]:
####  Step 5: DTM   ####

import arcpy

arcpy.env.workspace = r'C:/Your path/LASArcpy'
in_lasd = r'C:/Your path/LASArcpy/Extract_CathedralPines_2016.lasd'
out_raster = r'C:/Your path/DTM_CathedralPines_2016.tif'

# Create a temporary LAS dataset layer with only ground points
ground_layer = "ground_layer"
arcpy.management.MakeLasDatasetLayer(
    in_lasd,
    ground_layer,
    class_code="2",  # Class code 2 is typically for ground points
    return_values="",
    no_flag="INCLUDE_UNFLAGGED",
    synthetic="INCLUDE_SYNTHETIC",
    keypoint="INCLUDE_KEYPOINT",
    withheld="EXCLUDE_WITHHELD")

# Using the older function syntax
arcpy.LasDatasetToRaster_conversion(
    in_lasd,
    out_raster,
    "ELEVATION",
    "BINNING MINIMUM LINEAR",
    "FLOAT",
    "CELLSIZE",
    0.3,
    1)

print("DTM created successfully.")




In [ ]:
#### Step 6: CHM #####

import arcpy
from arcpy.sa import *

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

arcpy.env.workspace = r'C:/Your path/CHM'
dsm_raster = r'C:/Your path/DSM_CathedralPines_2016.tif'
dtm_raster = r'C:/Your path/DTM_CathedralPines_2016.tif'
chm_raster = r'C:/Your path/CHM_CathedralPines_2016.tif'

# Calculate CHM (DSM - DTM)
chm = Raster(dsm_raster) - Raster(dtm_raster)

# Set negative values to zero (optional but recommended)
chm = Con(chm < 0, 0, chm)

# Save the CHM
chm.save(chm_raster)

# Check in the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")

print("CHM created successfully.")


In [ ]:
#### Step 7: Intensity #####

import arcpy

arcpy.env.workspace = r'C:/Your path/CathedralPines'
in_lasd = r'C:/Your path/Extract_CathedralPine_2016.lasd'
out_intensity = r'C:/Your path/Intensity_CathedralPine2016.tif'


# Create intensity raster
arcpy.LasDatasetToRaster_conversion(
    in_lasd,
    out_intensity,
    "INTENSITY",  # Use INTENSITY instead of ELEVATION
    "BINNING AVERAGE LINEAR",  # Average intensity in each cell
    "FLOAT",
    "CELLSIZE",
    0.3,
    1)

print("Intensity raster created successfully.")
arcpy.CheckInExtension("Spatial")


Intensity raster created successfully.


'CheckedOut'

In [ ]:
#### Step 8: Density #####

import arcpy

# Set workspace and input/output paths
arcpy.env.workspace = r'C:/Your path/CathPines'
in_lasd = r'C:/Your path/Extract_CathedralPine_2016.lasd'
out_density = r'C:/Your path/Density_CathedralPine2016.tif'

try:
    # Use the LAS Point Statistics As Raster tool from Data Management
    arcpy.management.LasPointStatsAsRaster(
        in_lasd,
        out_density,
        "POINT_COUNT",  # Use POINT_COUNT for density
        "CELLSIZE",
        0.3)  # Cell size matching your other rasters
    
    print("Density raster created successfully using LasPointStatsAsRaster.")
except Exception as e:
    print(f"Failed to create density raster: {str(e)}")
    
    # Try alternative tool name if the first one fails
    try:
        arcpy.management.LasPointStatisticsAsRaster(
            in_lasd,
            out_density,
            "POINT_COUNT",
            "CELLSIZE",
            0.3)
        
        print("Density raster created successfully using LasPointStatisticsAsRaster.")
    except Exception as e2:
        print(f"Alternative method also failed: {str(e2)}")
        print("Please check that you have the required extensions enabled.")
